In [219]:
final_columns = ['march_min_temp', 'march_avg_temp', 'march_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'floor_area', 'ELEVATION',
       'energy_star_rating', 'days_above_90F',
       'precipitation_inches', 'snowdepth_inches', 'days_below_0F',
       'days_above_100F', 'days_above_110F', 'year_built', 'Year_Factor',
       'State_Factor', 'building_class', 'facility_type']

In [220]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor
ss= StandardScaler()
from sklearn.preprocessing import LabelEncoder
from category_encoders import TargetEncoder

In [221]:
dataset = pd.read_csv('../Datasets/x_test.csv')

In [222]:
y = pd.read_csv('../Datasets/y_test.csv')

In [223]:
y.drop('id', axis=1, inplace=True)
y

,site_eui
0,166.588554
1,259.381565
2,158.537090
3,261.441520
4,242.967711
...,...
9700,23.677329
9701,20.512284
9702,39.285975
9703,29.529438


In [224]:
dataset = dataset[final_columns]

In [225]:
dataset.head()

,march_min_temp,march_avg_temp,march_max_temp,august_min_temp,august_avg_temp,august_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,...,precipitation_inches,snowdepth_inches,days_below_0F,days_above_100F,days_above_110F,year_built,Year_Factor,State_Factor,building_class,facility_type
0,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,25.24,0,0,2,0,1994.0,7,State_1,Commercial,Grocery_store_or_food_market
1,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,25.24,0,0,2,0,1961.0,7,State_1,Commercial,Grocery_store_or_food_market
2,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,25.24,0,0,2,0,1950.0,7,State_1,Commercial,Grocery_store_or_food_market
3,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,25.24,0,0,2,0,1971.0,7,State_1,Commercial,Grocery_store_or_food_market
4,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,25.24,0,0,2,0,1942.0,7,State_1,Commercial,Grocery_store_or_food_market


In [226]:
dataset = pd.concat([dataset, y], axis=1)

In [227]:
dataset.head()

,march_min_temp,march_avg_temp,march_max_temp,august_min_temp,august_avg_temp,august_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,...,snowdepth_inches,days_below_0F,days_above_100F,days_above_110F,year_built,Year_Factor,State_Factor,building_class,facility_type,site_eui
0,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1994.0,7,State_1,Commercial,Grocery_store_or_food_market,166.588554
1,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1961.0,7,State_1,Commercial,Grocery_store_or_food_market,259.381565
2,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1950.0,7,State_1,Commercial,Grocery_store_or_food_market,158.537090
3,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1971.0,7,State_1,Commercial,Grocery_store_or_food_market,261.441520
4,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1942.0,7,State_1,Commercial,Grocery_store_or_food_market,242.967711


In [228]:
dataset.shape

(9705, 30)

#### Cleaning Dataset

In [194]:
def cleanStarRatings(final_df):
  selected_features =['march_min_temp', 'march_avg_temp', 'march_max_temp', 'august_min_temp',
       'august_avg_temp', 'august_max_temp', 'october_min_temp',
       'october_avg_temp', 'october_max_temp', 'november_min_temp',
       'november_avg_temp', 'november_max_temp', 'december_min_temp',
       'december_avg_temp', 'december_max_temp', 'floor_area', 'ELEVATION',
       'days_above_90F', 'site_eui', 'precipitation_inches',
       'snowdepth_inches', 'days_below_0F', 'days_above_100F',
       'days_above_110F', 'year_built']
  
  # Cleaning START ==============================================================================================
  final_df = final_df[selected_features]
  non_null_features = final_df.drop('energy_star_rating', axis=1).columns
  non_null_indexes= final_df[~ pd.isna(final_df['energy_star_rating'])].index
  null_indexes = final_df[pd.isna(final_df['energy_star_rating'])].index
  trainingDataFrame = final_df[non_null_features].iloc[non_null_indexes]
  target_y = final_df['energy_star_rating'].iloc[non_null_indexes]
  testDataFrame = final_df[non_null_features].iloc[null_indexes]
  trainingDataFrame = ss.fit_transform(trainingDataFrame)
  X_train, X_test, y_train, y_test = train_test_split(trainingDataFrame, target_y, test_size=.3, random_state=42)
  knn = KNeighborsRegressor(n_neighbors=15)
  knn.fit(X_train, y_train)
  mean_squared_error(y_test, knn.predict(X_test), squared=False)
  prediction = [int(x) for x in knn.predict(ss.fit_transform(testDataFrame))]
  predictedDataFrame = pd.DataFrame(prediction, columns=['energy_star_rating'], index=null_indexes)
  for i in null_indexes:
    final_df['energy_star_rating'][i] = predictedDataFrame['energy_star_rating'][i]
  # Cleaning END ==================================================================================================

  return final_df

In [229]:
def imputeYear(value):
  if value == 0.0 or pd.isna(value):
    return 2030
  else:
    return value

In [230]:
def clean(final_df):
  #Year
  final_df['year_built'] = final_df['year_built'].apply(imputeYear)

  #Energy Star Rating
  final_df.dropna(inplace=True)
  #final_df = cleanStarRatings(final_df)
  return final_df

In [231]:
dataset = clean(dataset)

In [232]:
dataset.shape

(7451, 30)

In [233]:
def impute110(value):
  if value == 0:
    return 0
  else:
    return 1

In [234]:
def encodeYear(value):
  if 1600.0 <= value <= 1959.0:
    return "1600-1959"
  elif 1960.0 <= value <= 2039.0:
    return "1960-2039"

In [235]:
def cleanYear(value):
  if 1600.0 <= value <= 1899.0:
    return "Ancient"
  else:
    return value

In [236]:
def imputeSD(value):
  if 0<= value <= 99:
    return "0-99"
  elif 100 <= value <= 299:
    return "100-299"
  else :
    return "300-1300"


In [237]:
def encodeFacilityType(value):
  if value not in ['Multifamily_Uncategorized', 'Office_Uncategorized']:
    return 'Others'
  else:
    return value

In [238]:
def encode(final_df):
  cat_features = ['Year_Factor', 'State_Factor', 'building_class', 'facility_type']
  encodingMapper = final_df[cat_features].copy()
  #Days
  final_df['days_above_110F'] = final_df['days_above_110F'].apply(impute110)
  final_df['days_above_100F'] = final_df['days_above_100F'].apply(impute110)
  final_df['days_below_0F'] = final_df['days_below_0F'].apply(impute110)

  #Year Built
  final_df['year_built'] = final_df['year_built'].apply(cleanYear)
  final_df = final_df[final_df['year_built'] !='Ancient']

  #Snow Depth Inches
  final_df['snowdepth_inches'] = final_df['snowdepth_inches'].apply(imputeSD)
  # encoder = TargetEncoder()
  # encodingMapper['snowdepth_inches'] = encoder.fit_transform(final_df['snowdepth_inches'], final_df['site_eui'])
  # final_df['snowdepth_inches'] = encodingMapper['snowdepth_inches']
  encoder = LabelEncoder()
  final_df['snowdepth_inches'] = encoder.fit_transform(final_df['snowdepth_inches'])
  #final_df['snowdepth_inches'] = encodingMapper['snowdepth_inches']

  #Facility Type
  final_df['facility_type'] = final_df['facility_type'].apply(encodeFacilityType)

  # State_Factor, building_class, facility_type, Year_Factor
  cats = ['State_Factor', 'building_class', 'facility_type', 'Year_Factor']
  for col in cats:
      encoder = LabelEncoder()
      final_df[col] = encoder.fit_transform(final_df[col])
      #final_df[col] = encodingMapper[f'te_{col}']
    
  return final_df

In [239]:
dataset.head()

,march_min_temp,march_avg_temp,march_max_temp,august_min_temp,august_avg_temp,august_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,...,snowdepth_inches,days_below_0F,days_above_100F,days_above_110F,year_built,Year_Factor,State_Factor,building_class,facility_type,site_eui
0,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1994.0,7,State_1,Commercial,Grocery_store_or_food_market,166.588554
1,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1961.0,7,State_1,Commercial,Grocery_store_or_food_market,259.381565
2,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1950.0,7,State_1,Commercial,Grocery_store_or_food_market,158.537090
3,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1971.0,7,State_1,Commercial,Grocery_store_or_food_market,261.441520
4,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,2,0,1942.0,7,State_1,Commercial,Grocery_store_or_food_market,242.967711


In [240]:
dataset = encode(dataset)

C:\Users\Priyesh Dave\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [241]:
dataset.head()

,march_min_temp,march_avg_temp,march_max_temp,august_min_temp,august_avg_temp,august_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,...,snowdepth_inches,days_below_0F,days_above_100F,days_above_110F,year_built,Year_Factor,State_Factor,building_class,facility_type,site_eui
0,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,1,0,1994.0,0,0,0,2,166.588554
1,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,1,0,1961.0,0,0,0,2,259.381565
2,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,1,0,1950.0,0,0,0,2,158.537090
3,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,1,0,1971.0,0,0,0,2,261.441520
4,42,56.935484,77,53,65.854839,88,48,64.193548,91,43,...,0,0,1,0,1942.0,0,0,0,2,242.967711


In [242]:
dataset.shape

(7329, 30)

In [209]:
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [243]:
X = dataset.drop('site_eui', axis=1)
y= dataset['site_eui']

In [244]:
X.shape

(7329, 29)

In [245]:
y.shape

(7329,)

In [251]:
X_Val, X_test, y_Val, y_test = train_test_split(X,y, test_size=.3, random_state=42)

In [252]:
def predict(model):
  predictions_train = model.predict(X_Val)
  predictions_test = model.predict(X_test)

  print(r2_score(y_Val,predictions_train))
  print(r2_score(y_test,predictions_test))
  print('RMSE: ',mean_squared_error(y_test, predictions_test, squared=False))


In [248]:
import joblib

In [249]:
catBoost = joblib.load('../Model/catBoost.pkl')
grid_search = joblib.load('../Model/gridSearch.pkl')